# Import necessary modules and create a spark session

In [27]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,sum,count
spark = SparkSession.builder.getOrCreate()

# Reading the Dataset from csv

In [5]:
file_path = 'OnlineRetail.csv'
df = spark.read.csv(file_path, header=True, inferSchema=True)
df.show(5)
df.printSchema()

+---------+---------+--------------------+--------+----------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|     InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+----------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|01-12-2010 08:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|01-12-2010 08:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|01-12-2010 08:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|01-12-2010 08:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|01-12-2010 08:26|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+----------------+---------+----------+--------------+
only showing top 5 rows

root
 |-- InvoiceNo: string (n


# Data Cleaning


### number of rows and columns

In [7]:
num_rows = df.count()
num_cols = len(df.columns)
print(f"Shape: ({num_rows}, {num_cols})")

Shape: (541909, 8)


### Check for null values in each column and sum them

In [12]:
null_counts = {c: df.where(col(c).isNull()).count() for c in df.columns}
print("Null counts per column:")
for col, count in null_counts.items():
    print(f"{col}: {count}")

Null counts per column:
InvoiceNo: 0
StockCode: 0
Description: 1454
Quantity: 0
InvoiceDate: 0
UnitPrice: 0
CustomerID: 135080
Country: 0


### Dropping rows with any null values

In [15]:
df = df.na.drop()

### Show the updated DataFrame schema and shape after dropping null values

In [16]:
df.printSchema()
num_rows_no_null = df.count()
num_cols_no_null = len(df.columns)
print(f"Shape after dropping null values: ({num_rows_no_null}, {num_cols_no_null})")

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)

Shape after dropping null values: (406829, 8)


# Data preprocessing 

### Total amount of purchase done by each individual customer

In [25]:
from pyspark.sql.types import StringType
df = df.withColumn('CustomerID', df['CustomerID'].cast(StringType()))
df = df.withColumn('Amount', df['Quantity'] * df['UnitPrice'])
rfm_ds_m = df.groupBy('CustomerID').agg(sum('Amount').alias('Amount'))
rfm_ds_m = rfm_ds_m.orderBy('CustomerID')
rfm_ds_m.show(10)

+----------+------------------+
|CustomerID|            Amount|
+----------+------------------+
|     12346|               0.0|
|     12347|            4310.0|
|     12348|           1797.24|
|     12349|           1757.55|
|     12350|334.40000000000003|
|     12352|1545.4100000000003|
|     12353|              89.0|
|     12354|            1079.4|
|     12355|             459.4|
|     12356|2811.4300000000003|
+----------+------------------+
only showing top 10 rows



### Calculate Frequency (number of transactions) per CustomerID

In [30]:
rfm_ds_f = df.groupBy('CustomerID').agg(count('InvoiceNo').alias('Frequency'))
rfm_ds_f = rfm_ds_f.orderBy('CustomerID')
rfm_ds_f.show(10)

+----------+---------+
|CustomerID|Frequency|
+----------+---------+
|     12346|        2|
|     12347|      182|
|     12348|       31|
|     12349|       73|
|     12350|       17|
|     12352|       95|
|     12353|        4|
|     12354|       58|
|     12355|       13|
|     12356|       59|
+----------+---------+
only showing top 10 rows

